# Task 1:  Day-ahead energy forecasting

## General Setting

Electricity, till date, cannot be stored in large amounts, therefore supply and demand need to always be balanced by the energy providers. The accurate short term forecast of energy demand is critical for the operations and control of productive capacity, with significant consequences. However, as with any forecast, there is typically some uncertainty involved. This uncertainty is especially heightened in the case of energy forecasting today, where alternate sources of energy such as solar panels are ubiquitous. Also, with the transition to e-mobility additional non-traditional consumer patterns contribute to the forecasting uncertainty. Therefore, understanding electricity consumption behaviour either for individual households or for regional groups of households becomes key for the future electricity market.

## Task

This data science challenge task entails estimating energy use forecasts for upto a week, for groups of households in the UK energy market, selected based on geographical similarity. The challenge has two sub-tasks-
1. the first sub-task, only one value for the single day ahead is required to be estimated. In other words the aggregated use for one day (1 value per day).
2. In the second sub-task, the demand for each hour in the day-ahead is to be estimated (24 values per day).

You are provided with historical half-hourly energy readings for the 61 anonymised groups between 1 January 2017 and 04 September 2019. A week is sliced off from each 45-day window and reserved for testing the models. You are required to estimate these missing periods in the two frequencies.

Every group consists of a different number of dwellings, for which energy consumption profile has been summed up for two reasons: data privacy and forecasting accuracy.

All data is provided in csv format and described below. We also provide code snippets for loading the data and creating submission files.


The data is divided into two phases for evaluation purposes. The development set can be used in phase 2, and the final test will be made available in phase 3.

The Development set contains test dates from 08/02/2017 until 21/03/2019. (Train set: 01/01/2017 - 31/03/2019)
The test set contains dates from 24/04/209 until 04/09/2019.            (Train set: 01/04/2019 - 28/08/2019)

Each phase will require submissions for both sub tasks for the respective dates.

##  Data 

`train.csv`: contains the data values in KWh at a half-hourly frequency for the 61 different groups.

<b>Column Description</b>:

`pseudo_id`: Anonymised IDs for dwelling groups (string).

$a_{ij}$: Energy consumption for household $i$ between timestep $j$ and $j+1$ (float64). </br>

For e.g.: </br>

`2017-01-01 00:00:00` indicates electricity consumption in KWh between 2017-01-01 00:00:00 and 2017-01-01 00:30:00. 

In [3]:
# Imports
import numpy as np
import pandas as pd
from datetime import datetime
from dataset import Dataset

### Data snapshot

In [4]:
# Load csv in pandas, using 'pseudo_id' as index
train_df = pd.read_csv('train.csv', index_col = 'pseudo_id')

In [5]:
# Print first few lines in train_df
train_df.head()

,2017-01-01 00:00:00,2017-01-01 00:30:00,2017-01-01 01:00:00,2017-01-01 01:30:00,2017-01-01 02:00:00,2017-01-01 02:30:00,2017-01-01 03:00:00,2017-01-01 03:30:00,2017-01-01 04:00:00,2017-01-01 04:30:00,...,2019-08-28 19:00:00,2019-08-28 19:30:00,2019-08-28 20:00:00,2019-08-28 20:30:00,2019-08-28 21:00:00,2019-08-28 21:30:00,2019-08-28 22:00:00,2019-08-28 22:30:00,2019-08-28 23:00:00,2019-08-28 23:30:00
pseudo_id,,,,,,,,,,,,,,,,,,,,,
0x16cb02173ebf3059efdc97fd1819f14a2,45.023,39.9850,36.5695,34.7480,35.972,38.439,36.591,36.3155,32.6605,0.1420,...,24.2880,23.994,26.1995,25.0270,23.0665,26.093,23.4295,25.4715,26.246,22.6020
0x1c9d08cd16fce04790ef900695861e786,2.931,1.6410,2.2600,2.2730,2.651,3.137,2.532,3.1420,2.5280,0.0000,...,2.5700,1.446,1.5230,1.5630,2.5880,2.190,1.4860,2.5270,2.288,1.7940
0x1612e4cbe3b1b85c3dbcaeaa504ee8424,11.014,12.6525,10.8240,13.7485,12.383,12.342,13.413,11.4840,11.5105,0.0455,...,6.3565,5.766,5.4955,5.0885,6.8140,7.492,5.7705,6.8240,6.072,6.7205
0x20158d36236a640cf0524dba149459169,55.813,49.0400,49.0950,41.1330,45.660,48.477,50.539,45.7370,42.6800,0.0000,...,32.6460,30.439,30.2470,31.2660,34.3390,33.076,33.1080,33.7260,30.009,34.8400
0xc305005dcb1ed6128d816954c5ab9e7e,26.925,28.1180,25.6000,28.0910,26.530,23.858,26.556,27.7140,23.1740,0.0000,...,13.3980,13.280,13.7340,13.6060,14.7000,16.290,15.1240,15.3650,14.360,13.9350


Note that the data points are given at a frequency resolution of 30 minutes. However, the task requires predictions either as daily or hourly aggregates. A week is missing after each consecutive 38 days (in total a cycle of 45 days), for instance after `2017-02-07` timestams are missing until `2017-02-14`. That means values for those dates are missing and are to be predicted.

### Fit a Dummy Regressor

Example for Subtask 1: Day-ahead daily forecast

In [6]:
# read csv
df = pd.read_csv('train.csv')
# drop index for feature preparation
df_ = df.drop(columns='pseudo_id')
# convert dates to pandas datetime
df_.columns = [datetime.strptime(c, "%Y-%m-%d %H:%M:%S") for c in df_.columns]
# Aggregate energy use values per day
df_ = df_.T.groupby(df_.T.index.date).sum()
# Set dates for development phase
new_date_range = pd.date_range(start="2017-01-01", end="2019-03-31", freq="D")
# Add test dates in the data frame
df_ = df_.reindex(new_date_range, fill_value = 100) # using dummy values in test set
# df_ = df_.T

df_['date'] = df_.index
df_.head()

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,date
2017-01-01,1673.8655,143.162,586.9155,2094.588,1260.608,432.268,211.028,1191.6520,1456.7335,503.095,...,313.948,196.943,36.989,153.074,19.533,80.263,175.5120,40.8825,112.112,2017-01-01
2017-01-02,1463.3230,130.557,532.9330,1845.933,1024.438,361.388,158.281,1022.2485,1319.9650,356.898,...,190.435,153.460,42.530,104.633,20.703,69.607,140.6670,31.2955,92.511,2017-01-02
2017-01-03,1132.2910,99.234,407.0630,1475.750,725.995,279.199,125.445,765.0000,991.8735,257.116,...,110.994,88.819,33.476,79.647,14.527,38.887,108.7145,29.8745,64.486,2017-01-03
2017-01-04,1183.8085,91.932,325.4720,1553.638,702.530,306.612,119.487,741.4780,1011.5705,278.864,...,55.876,78.585,30.363,75.137,16.648,53.533,117.0440,33.0020,54.094,2017-01-04
2017-01-05,1125.2870,76.781,328.4645,1572.033,672.427,277.665,116.686,677.7000,1038.2905,284.385,...,57.125,79.498,30.640,76.458,17.598,48.185,119.8990,37.0340,48.022,2017-01-05


In [7]:
from ts_split import GroupedTimeSeriesSplit
from models import SeasonalPredictor
# Prepare data into train/test split
tscv = GroupedTimeSeriesSplit(train_window= 38, test_window=7, train_gap = 0)
## Use your Scikit estimator
# est = your_estimator()
est = SeasonalPredictor()
# perform prediction
predictions = []
date_col = 61
for train_ind, test_ind in tscv.split(df_, y=df_, dates = df_.index):
    X_train = df_.iloc[train_ind]
    y_train = df_.iloc[train_ind].shift(1)
    est = est.fit(X_train, y_train)
    X_test = df_.iloc[test_ind].copy()
    ## predict on the test dataframe
    X_test = est.predict(X_test)
    prediction_df = pd.DataFrame(X_test)
    prediction_df[date_col] = prediction_df[date_col] + pd.to_timedelta(8, unit= 'd')
    predictions.append(prediction_df)
predictions = pd.concat(predictions)
predictions.index = predictions[date_col]
predictions.drop(columns = [date_col], inplace = True)
predictions = predictions.T.iloc[:, :-3]
# add original pseudo_ids
#predictions = predictions.set_index(df.pseudo_id)

predictions.insert(0, 'pseudo_id', df.pseudo_id)

In [8]:
predictions

61,pseudo_id,2017-02-08 00:00:00,2017-02-09 00:00:00,2017-02-10 00:00:00,2017-02-11 00:00:00,2017-02-12 00:00:00,2017-02-13 00:00:00,2017-02-14 00:00:00,2017-03-25 00:00:00,2017-03-26 00:00:00,...,2019-02-02 00:00:00,2019-02-03 00:00:00,2019-02-04 00:00:00,2019-03-15 00:00:00,2019-03-16 00:00:00,2019-03-17 00:00:00,2019-03-18 00:00:00,2019-03-19 00:00:00,2019-03-20 00:00:00,2019-03-21 00:00:00
0,0x16cb02173ebf3059efdc97fd1819f14a2,1519.776,1372.3825,1056.1095,1290.8265,1737.6175,1935.3125,1807.4465,1578.1475,1744.169,...,4282.343,2920.512,2406.4675,3588.6385,2612.024,2502.003,2629.7525,2483.6425,3274.512,3637.6735
1,0x1c9d08cd16fce04790ef900695861e786,89.082,82.68,75.174,81.824,113.631,128.987,107.259,105.664,110.331,...,282.708,207.484,165.91,256.783,193.636,185.978,168.902,162.666,224.949,268.944
2,0x1612e4cbe3b1b85c3dbcaeaa504ee8424,389.7045,290.5195,241.494,334.4945,390.1805,490.0385,424.5275,503.898,579.7995,...,819.7645,538.4465,370.7955,750.352,431.749,401.8295,349.459,375.7885,728.0365,831.205
3,0x20158d36236a640cf0524dba149459169,3238.623,2919.566,2355.022,2990.302,4014.421,4458.047,4083.174,2969.472,3246.188,...,4732.771,3397.678,2738.874,4191.467,2963.118,2846.125,3065.533,2839.945,3819.749,4140.033
4,0xc305005dcb1ed6128d816954c5ab9e7e,624.56,584.828,504.496,590.362,948.21,1071.587,1000.546,873.621,931.245,...,2334.2,1922.352,1534.155,2192.406,1663.886,1588.094,1616.771,1495.21,2015.257,2178.948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0x12342fbadc0ca9418f2d540bb3cb8364a,20.653,17.715,12.516,15.149,22.64,31.045,22.573,25.179,27.049,...,61.599,40.52,39.822,61.912,45.559,48.264,53.415,58.642,67.357,68.06
57,0x16d1816bc5d185c47de080d4c6a64bc9a,51.941,60.318,45.359,45.105,54.483,60.591,55.662,48.201,52.731,...,169.361,123.678,100.643,145.036,124.183,108.917,122.937,109.285,123.424,147.961
58,0x14f480f24c435af1b8574c1c6bab38a1c,111.7475,102.0545,86.983,83.047,124.799,154.621,134.693,137.511,154.07,...,446.0455,279.696,221.3995,324.4705,216.985,208.4295,233.2085,208.3125,303.62,355.8485
59,0x1c7fc724d0a4f89ed1de8a0a4b302db22,79.7915,73.215,55.783,65.838,41.029,45.3185,37.4015,69.1915,80.2005,...,117.579,95.319,71.176,98.457,75.0605,65.3685,73.8055,68.3145,90.101,113.81


### Train a Pytorch model

Load Custom Pytorch Dataset
Please refer to `dataset.py` for the source code

Features used:
- N days lag
- DateTime features, i.e month, season, day of the year, etc.

In [9]:
# Create an instance of Dataset class, sampled at hourly frequency, using features with 7 day lag
# Frequency: D -> Daily, H -> Hourly
dataset = Dataset('train.csv', frequency= "D", window_size= 7)

In [10]:
# Visualise a sample
print(dataset[1][0].shape)

torch.Size([815, 14])


In [11]:
from models import MLP, train_model #simple PyTorch MLP model, see models.py
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

# Some hyperparameters
params = {
    'batch_size' : 256,
    'lr' : 0.001,
    'num_epochs' : 10

    }

input_dim = dataset[0][0].size(1)

# Define dataloader
data = DataLoader(dataset, batch_size=params['batch_size'], shuffle=False, sampler=None, batch_sampler=None, num_workers=0,
                  collate_fn=None, pin_memory=True, drop_last=False)


# Assign device for computation
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# init model
model = MLP(input_dim)

# Run training
train_model(model, data, device, **params)

MLP(
  (fc): Linear(in_features=14, out_features=64, bias=True)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (activation): ReLU()
)
epoch:   0 loss: 683908.31250000
epoch:   1 loss: 634419.50000000
epoch:   2 loss: 576422.87500000
epoch:   3 loss: 518585.43750000
epoch:   4 loss: 461839.71875000
epoch:   5 loss: 406354.34375000
epoch:   6 loss: 352179.62500000
epoch:   7 loss: 299533.65625000
epoch:   8 loss: 248835.51562500
epoch:   9 loss: 200714.21875000
Finished training and model saved!


## Submission of Results

For each `pseudo_id` given in the train set, you are required to predict the energy consumption for each missing day in the follwing two formats:

1. Subtask 1: Aggregated usage values for each day, i.e., 1 value per day, and
2. Subtask 2: Hourly usage values for each day, i.e., 24 values per day.

Both files are required in the submission file to calculate evaluation metrics.

In [12]:
# print test indices
dataset.get_test_idx()

DatetimeIndex(['2017-02-08', '2017-02-09', '2017-02-10', '2017-02-11',
               '2017-02-12', '2017-02-13', '2017-02-14', '2017-03-25',
               '2017-03-26', '2017-03-27',
               ...
               '2019-02-02', '2019-02-03', '2019-02-04', '2019-03-15',
               '2019-03-16', '2019-03-17', '2019-03-18', '2019-03-19',
               '2019-03-20', '2019-03-21'],
              dtype='datetime64[ns]', length=126, freq=None)

A prediction for each time step in respective frequencies is to be calculated and stored as csv.

### Example Submission 

Subtask 1:


<code> ```pseudo_id, 2017-02-08 00:00:00, 2017-02-09 00:00:00, 2017-02-10 00:00:00, ........, 2019-02-28 00:00:00
0xd05, 23.4, 11.3, 23.2, ......., 32.4 
0xd06, 21.4, 21.3, 13,2, ......., 42.4```</code>



Subtask 2:


<code>```pseudo_id, 2017-02-08 00:00:00 , 2017-02-08 01:00:00, 2017-02-08 02:00:00, ........, 2019-02-28 23:00:00
0xd05, 3.4, 1.3, 2.2, ......., 3.4
0xd06, 1.4, 1.3, 3.2, ......., 1.4```</code>

In [13]:
sample_submission_hourly = pd.read_csv('sample_submission_hourly.csv')
sample_submission_daily = pd.read_csv('sample_submission_daily.csv')


In [14]:
sample_submission_daily

,pseudo_id,2017-02-08 00:00:00,2017-02-09 00:00:00,2017-02-10 00:00:00,2017-02-11 00:00:00,2017-02-12 00:00:00,2017-02-13 00:00:00,2017-02-14 00:00:00,2017-03-25 00:00:00,2017-03-26 00:00:00,...,2019-02-02 00:00:00,2019-02-03 00:00:00,2019-02-04 00:00:00,2019-03-15 00:00:00,2019-03-16 00:00:00,2019-03-17 00:00:00,2019-03-18 00:00:00,2019-03-19 00:00:00,2019-03-20 00:00:00,2019-03-21 00:00:00
0,0x16cb02173ebf3059efdc97fd1819f14a2,13.987208,29.553732,47.619200,143.206933,58.411633,43.562970,64.988182,26.708951,20.661111,...,48.926269,59.032178,24.770144,280.487625,125.79070,161.283800,26.490250,292.013091,90.337917,206.394079
1,0x1c9d08cd16fce04790ef900695861e786,1.084910,2.081561,3.002167,8.366533,3.490800,2.360909,3.558864,1.423885,1.132635,...,3.058385,4.284467,1.758928,19.664917,8.53730,10.815800,1.505930,18.284182,4.755024,11.853368
2,0x1612e4cbe3b1b85c3dbcaeaa504ee8424,4.007826,7.907780,13.610350,43.034500,19.164833,15.053394,20.003545,11.056025,7.523532,...,9.183250,10.484478,3.892851,53.468083,18.81305,19.575667,3.336262,53.663591,21.389083,48.687237
3,0x20158d36236a640cf0524dba149459169,35.230236,67.194805,98.574933,264.427533,113.122500,72.965606,100.897636,36.520836,24.231635,...,55.496154,70.756578,29.199443,331.659167,149.62290,186.642200,30.403372,347.752273,110.067190,255.272000
4,0xc305005dcb1ed6128d816954c5ab9e7e,6.789202,14.484000,22.991033,66.257067,27.345000,20.335576,31.606682,13.573951,10.192111,...,28.015628,36.832333,14.771062,163.655250,74.80040,89.186800,14.195151,166.189636,51.751048,136.172263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0x12342fbadc0ca9418f2d540bb3cb8364a,0.186472,0.395049,0.545700,1.245867,0.558533,0.424848,0.712136,0.235967,0.203857,...,0.821346,1.058489,0.491835,6.144167,2.48965,3.149533,0.581709,5.158273,1.958167,4.398000
57,0x16d1816bc5d185c47de080d4c6a64bc9a,0.614483,1.155976,1.773367,4.009533,1.747833,1.528424,2.513273,0.845934,0.783413,...,2.142833,2.745089,1.192990,11.314083,6.16480,8.082533,1.244198,12.741000,3.843762,8.608105
58,0x14f480f24c435af1b8574c1c6bab38a1c,1.125899,2.497805,3.379600,9.475667,3.852150,2.887364,4.847750,1.659303,1.392317,...,4.793026,6.137600,2.127325,24.964417,11.06180,12.624133,2.421535,24.984000,8.677738,21.057737
59,0x1c7fc724d0a4f89ed1de8a0a4b302db22,0.708573,1.508341,1.753900,2.599433,1.295650,1.320864,2.056591,0.534238,0.472079,...,1.810468,2.608211,0.619160,8.501042,4.69080,3.800633,0.596297,6.087864,2.326810,5.772895


## Evaluation


The evaluation metric for this competition is Mean Absolute Percentage Error, computed as:

$\text{MAPE} = \frac{1}{T*G} \sum_{g} (\sum_{t} |\frac{a_{t,g}-f_{t,g}}{a_{t,g}}|)  \times 100$ where,

$T$: number of timesteps (days for sub-task 1 and hours for sub-task 2)</br>
$G$: number of groups (days for sub-task 1 and hours for sub-task 2)</br>
$a_{i,j}$: actual value for time point t of group g </br>
$f_{i,j}$: forecasted value for time point t of group g</br>

We will weight the MAPE for both sub-tasks equally. MAPE averaged over all the timesteps over all the days will be the final score.

 The lower the final score, the better the forecast

In [15]:
from utils import evaluate
# Example MAPE computation
# evaluate(y, y_hat)
# shapes must match otherwise it'll throw assertion error
evaluate(sample_submission_daily, predictions)
# nevermind the large error, the sample ground truth is arbitrary

5177.15963325088

### Prepare Submission
Submission files from each task can finally be zipped together

In [16]:
from utils import create_submission
create_submission(predictions, sample_submission_hourly)

wrote submission-2022-05-27_13-02-55.215280.zip


### Challenge Rules

1. The submission file must be a single zip containing submissions for both tasks as csv.

2. The csv files need to be in the following format:</i><b> N×[′pseudo_id′,timestamp_1(float),   timestamp_2(float),...,timestamp_24(float) ] </i></b>, where pseudo_id is the predicted group id.

3. The submission files must match the test indices and must be with the same shape as the sample submissions.

4. Participants are not permitted to use external data for system development. Any combination of feature engineering and modelling techniques given the data is permitted. Creating decision ensembles is encouraged.



In [17]:
from utils import evaluate
# Example MAPE computation
# evaluate(y, y_hat)
# shapes must match otherwise it'll throw assertion error
evaluate(sample_submission_daily, predictions)
# nevermind the large error, its to be expected with dummy estimators

5177.15963325088

### Prepare Submission
Submission files from each task can finally be zipped together

In [18]:
from utils import create_submission
create_submission(sample_submission_daily, sample_submission_hourly)

wrote submission-2022-05-27_13-02-55.388288.zip


### Challenge Rules

1. The submission file must be a single zip containing submissions for both tasks as csv.

2. The csv files need to be in the following format:</i><b> N×[′pseudo_id′,timestamp_1(float),   timestamp_2(float),...,timestamp_24(float) ] </i></b>, where pseudo_id is the predicted group id.

3. The submission files must match the test indices and must be with the same shape as the sample submissions.

4. Participants are not permitted to use external data for system development. Any combination of feature engineering and modelling techniques given the data is permitted. Creating decision ensembles is encouraged.



In [19]:
train_df.

,2017-01-01 00:00:00,2017-01-01 00:30:00,2017-01-01 01:00:00,2017-01-01 01:30:00,2017-01-01 02:00:00,2017-01-01 02:30:00,2017-01-01 03:00:00,2017-01-01 03:30:00,2017-01-01 04:00:00,2017-01-01 04:30:00,...,2019-08-28 19:00:00,2019-08-28 19:30:00,2019-08-28 20:00:00,2019-08-28 20:30:00,2019-08-28 21:00:00,2019-08-28 21:30:00,2019-08-28 22:00:00,2019-08-28 22:30:00,2019-08-28 23:00:00,2019-08-28 23:30:00
pseudo_id,,,,,,,,,,,,,,,,,,,,,
0x16cb02173ebf3059efdc97fd1819f14a2,45.023,39.9850,36.5695,34.7480,35.972,38.439,36.591,36.3155,32.6605,0.1420,...,24.2880,23.994,26.1995,25.0270,23.0665,26.093,23.4295,25.4715,26.246,22.6020
0x1c9d08cd16fce04790ef900695861e786,2.931,1.6410,2.2600,2.2730,2.651,3.137,2.532,3.1420,2.5280,0.0000,...,2.5700,1.446,1.5230,1.5630,2.5880,2.190,1.4860,2.5270,2.288,1.7940
0x1612e4cbe3b1b85c3dbcaeaa504ee8424,11.014,12.6525,10.8240,13.7485,12.383,12.342,13.413,11.4840,11.5105,0.0455,...,6.3565,5.766,5.4955,5.0885,6.8140,7.492,5.7705,6.8240,6.072,6.7205
0x20158d36236a640cf0524dba149459169,55.813,49.0400,49.0950,41.1330,45.660,48.477,50.539,45.7370,42.6800,0.0000,...,32.6460,30.439,30.2470,31.2660,34.3390,33.076,33.1080,33.7260,30.009,34.8400
0xc305005dcb1ed6128d816954c5ab9e7e,26.925,28.1180,25.6000,28.0910,26.530,23.858,26.556,27.7140,23.1740,0.0000,...,13.3980,13.280,13.7340,13.6060,14.7000,16.290,15.1240,15.3650,14.360,13.9350
